In [1]:
import pandas as pd
import numpy as np
import csv, sqlite3
import logging
import cProfile

In [2]:
### utilities

def _get_col_datatypes(fin):
    """from https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python """
    dr = csv.DictReader(fin) # comma is default delimiter
    fieldTypes = {}
    for entry in dr:
        feildslLeft = [f for f in dr.fieldnames if f not in fieldTypes.keys()]
        if not feildslLeft: break # We're done
        for field in feildslLeft:
            data = entry[field]

            # Need data to decide
            if len(data) == 0:
                continue

            if data.isdigit():
                fieldTypes[field] = "INTEGER"
            else:
                fieldTypes[field] = "TEXT"
        # TODO: Currently there's no support for DATE in sqllite

    if len(feildslLeft) > 0:
        raise Exception("Failed to find all the columns data types - Maybe some are empty?")

    return fieldTypes


def escapingGenerator(f):
    """from https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python """
    for line in f:
        yield line.encode("ascii", "xmlcharrefreplace").decode("ascii")


def csvToDb(csvFile, outputToFile = False, con=None, table_name=None):
    """ from https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python 
    with small changes """
    # TODO: implement output to file
    if table_name is None:
        table_name = 'ads'

    with open(csvFile,mode='r', encoding="ISO-8859-1") as fin:
        dt = _get_col_datatypes(fin)

        fin.seek(0)

        reader = csv.DictReader(fin)

        # Keep the order of the columns name just as in the CSV
        fields = reader.fieldnames
        cols = []

        # Set field and type
        for f in fields:
            cols.append("%s %s" % (f, dt[f]))

        # Generate create table statement:
        stmt = "CREATE TABLE " + table_name + " (%s)" % ",".join(cols)
        if con is None:
            print('Creating engine')
            con = sqlite3.connect(":memory:")
        cur = con.cursor()
        cur.execute(stmt)

        fin.seek(0)


        reader = csv.reader(escapingGenerator(fin))

        # Generate insert statement:
        stmt = "INSERT INTO " + table_name + " VALUES(%s);" % ','.join('?' * len(cols))

        cur.executemany(stmt, reader)
        con.commit()

    return con

class Profiler():
    """ ctxtM for old python without profile"""
    def __init__(self, nlines):
        self.nlines = nlines

    def __enter__(self):
        self.pr = cProfile.Profile()
        self.pr.enable()

    def __exit__(self, *args):
        import pstats, io
        self.pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(self.pr, stream=s).sort_stats(sortby)
        ps.print_stats(self.nlines)
        print(s.getvalue())
### end of utilities

In [3]:
number_of_test_subjects = 8
## pandas
with Profiler(2):
    client_info = pd.read_csv('../MOCK_DATA/mock_data_1.csv')
    male_clients = client_info['gender'] == 'Male'
    test_male_client = client_info[ male_clients ][:number_of_test_subjects]
    pd_out = test_male_client.ip_address

## sqlite3
with Profiler(2):
    con = sqlite3.connect(':memory:') # or file or ?
    con = csvToDb('../MOCK_DATA/mock_data_1.csv', con=con, table_name='client_info')
    query = """
    select ip_address
    from client_info
    where gender = 'Male'
    limit """ + str(number_of_test_subjects) + """
    """
    sql_out = con.execute(query)
    sql_label = [description[0] for description in sql_out.description]
    sql_data = sql_out.fetchall()

## now check
for i, (sql_x, pd_x) in enumerate(zip(sql_data, pd_out)):
    if i < number_of_test_subjects//2:
        print(sql_x[0], '|', pd_x)
    assert sql_x[0] == pd_x
print('     PASSED')

         2633 function calls (2610 primitive calls) in 0.006 seconds

   Ordered by: cumulative time
   List reduced from 346 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.003    0.003 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:503(parser_f)
        1    0.000    0.000    0.003    0.003 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:382(_read)



         423 function calls in 0.002 seconds

   Ordered by: cumulative time
   List reduced from 44 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-2-798a2db6f8e2>:35(csvToDb)
        1    0.000    0.000    0.001    0.001 {method 'executemany' of 'sqlite3.Cursor' objects}



61.75.73.233 | 61.75.73.233
140.135.125.62 | 140.135.125.62
198.116.8.254 | 198.116.8.254
240.193.252.193 | 240.19

In [4]:
client_info = pd.read_csv('../MOCK_DATA/mock_data_1.csv')
works_at = pd.read_csv('../MOCK_DATA/mock_data_2.csv')

# dumb pandas
with Profiler(2):
    print('email company name')
    print('------------------')
    for id_w, w in enumerate(works_at.email):
        for id_c, c in enumerate(client_info.email):
            if w == c:
                print(works_at.email[id_w], works_at.company[id_w], client_info.first_name[id_c])

email company name
------------------
keble29@usatoday.com Wordware Karly
dkobud@google.es Wikivu Durand
keble29@usatoday.com Dabshots Karly
         81050 function calls (81046 primitive calls) in 0.059 seconds

   Ordered by: cumulative time
   List reduced from 100 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1001    0.001    0.000    0.043    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:1049(__iter__)
     1001    0.002    0.000    0.039    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/dtypes/common.py:609(is_datetimelike)





In [5]:
## pandas
with Profiler(2):
    matches = works_at.email.isin(client_info.email)
    print('email company name')
    print('------------------')
    for email, company in zip( works_at[matches].email, works_at[matches].company):
        guy = client_info[client_info.email == email]
        print(email, company, guy.first_name.values[0])


email company name
------------------
keble29@usatoday.com Wordware Karly
dkobud@google.es Wikivu Durand
keble29@usatoday.com Dabshots Karly
         4440 function calls (4372 primitive calls) in 0.009 seconds

   Ordered by: cumulative time
   List reduced from 268 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    0.000    0.000    0.004    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:1940(__getitem__)
        5    0.000    0.000    0.003    0.001 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:1983(_getitem_array)





In [6]:
## sql
with Profiler(2):
    con = sqlite3.connect(':memory:') # or file or ?
    con = csvToDb('../MOCK_DATA/mock_data_1.csv', con=con, table_name='client_info')
    con = csvToDb('../MOCK_DATA/mock_data_2.csv', con=con, table_name='works_at')
    query = """
    select works_at.email, works_at.company, client_info.last_name
    from client_info, works_at
    where client_info.email = works_at.email
    """
    sql_out = con.execute(query)
    sql_label = [description[0] for description in sql_out.description]
    sql_data = sql_out.fetchall()
    print(sql_data)

[('email', 'company', 'last_name'), ('dkobud@google.es', 'Wikivu', 'Kobu'), ('keble29@usatoday.com', 'Dabshots', 'Eble'), ('keble29@usatoday.com', 'Wordware', 'Eble')]
         3499 function calls in 0.008 seconds

   Ordered by: cumulative time
   List reduced from 46 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.007    0.003 <ipython-input-2-798a2db6f8e2>:35(csvToDb)
        2    0.004    0.002    0.006    0.003 {method 'executemany' of 'sqlite3.Cursor' objects}



